| | |
|-|-|
| Author(s) |  [Guillaume Vernade](https://github.com/giom-v), [Eric Dong](https://github.com/gericdong) |

## Overview

[Gemini 2.0 Flash Thinking](https://cloud.google.com/vertex-ai/generative-ai/docs/thinking-mode) is an experimental model that's trained to generate the "thinking process" the model goes through as part of its response. As a result, the Flash Thinking model is capable of stronger reasoning capabilities in its responses than the Gemini 2.0 Flash model.

This tutorial demonstrates how to access the Gemini 2.0 Flash Thinking model and use the model to solve the following complex tasks that require multiple rounds of strategizing and iteratively solving.

- Example 1: Code simplification
- Example 2: Geometry problem (with image)
- Example 3: Mathematical brain teaser
- Example 4: Generating question for a specific level of knowledge
- Example 5: Statistics
- Example 6: Brain teaser with a twist


## Getting Started

### Install Google Gen AI SDK for Python


In [ ]:
%pip install --upgrade --quiet google-genai

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

### Import libraries


In [1]:
from collections.abc import Iterator
import os

from IPython.display import Image, Markdown, display
from google import genai
from google.genai.types import (
    GenerateContentConfig,
    GenerateContentResponse,
    Part,
    ThinkingConfig,
)

### Set Google Cloud project information and create client

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [14]:
PROJECT_ID = "[your-project-id]"  # @param {type: "string"}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

In [15]:
client = genai.Client(
    vertexai=True,
    project='jcoggshall-ml-2024',
    location='us-central1',
)

In [2]:
from creds import all_creds
client = genai.Client(api_key=all_creds['GEMINI_HIMS_API_KEY_mlproj_V1'], http_options={'api_version':'v1alpha'})

## Use Gemini 2.0 Flash Thinking Model


### Set model ID

See the [Google models](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models) page for more information.

In [15]:
MODEL_ID = "gemini-2.0-flash-thinking-exp-01-21"  # @param {type: "string"}

### Helper functions

Create methods to print out the thoughts and answer.

In [16]:
def print_thoughts(response: GenerateContentResponse) -> None:
    for part in response.candidates[0].content.parts:
        header = "Thoughts" if part.thought else "Answer"
        display(Markdown(f"""## {header}:\n{part.text}"""))


def print_thoughts_stream(response: Iterator[GenerateContentResponse]) -> None:
    display(Markdown("## Thoughts:\n"))
    answer_shown = False

    for chunk in response:
        for part in chunk.candidates[0].content.parts:
            if not part.thought and not answer_shown:
                display(Markdown("## Answer:\n"))
                answer_shown = True
            display(Markdown(part.text))

### Enable thoughts

You set the flag `include_thoughts` in the `ThinkingConfig` to indicate whether to return thoughts in the model response. The flag is set to `False` by default.

In [17]:
config = GenerateContentConfig(thinking_config=ThinkingConfig(include_thoughts=True))

### Generate content with thoughts

Then use the `generate_content` method to send a request to generate content with thoughts. The model responds with multiple parts, the thoughts and the model response. You can check the `part.thought` field to determine if a part is a thought or not.

In [18]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="What is the next number in this sequence: 2, 4, 8, 16, __?",
    config=config,
)
print_thoughts(response)

## Answer:
This sequence is a geometric sequence where each number is multiplied by 2 to get the next number.

* 2 * 2 = 4
* 4 * 2 = 8
* 8 * 2 = 16

To find the next number, we multiply the last number (16) by 2:

* 16 * 2 = 32

Therefore, the next number in the sequence is **32**.

In [19]:
print(response.candidates)

[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='This sequence is a geometric sequence where each number is multiplied by 2 to get the next number.\n\n* 2 * 2 = 4\n* 4 * 2 = 8\n* 8 * 2 = 16\n\nTo find the next number, we multiply the last number (16) by 2:\n\n* 16 * 2 = 32\n\nTherefore, the next number in the sequence is **32**.')], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=None, finish_reason=<FinishReason.STOP: 'STOP'>, grounding_metadata=None, index=0, logprobs_result=None, safety_ratings=None)]


### Generate content stream with thoughts

You can also use the `generate_content_stream` method to stream the response and thoughts as they are being generated, and the model will return chunks of the response as soon as they are generated.

In [7]:
response = client.models.generate_content_stream(
    model=MODEL_ID,
    contents="Does the Monty Hall Problem change if all the doors are made of transparent glass?",
    config=config,
)

print_thoughts_stream(response)

## Thoughts:


## Answer:


Yes, the Monty Hall Problem **changes significantly** if all the doors are made of transparent glass.  

In fact, it essentially **eliminates the paradox and the strategic element** of the original problem.

Here's why:

**In the Original Monty Hall Problem (Opaque Doors):**

* **Information Asymmetry:** You have limited information. You don't know where the car is. Monty *

does* know where the car is.
* **Monty's Action is Informative:** Monty opening a door to reveal a goat is not random. He *chooses* to open a door that he knows hides a goat and is *not* the door you initially picked. This action provides you with new information about the remaining

...

 will **always choose the door with the car behind it** on your first pick.  There's no reason to choose a door with a goat when you can clearly see the car.

2. **Monty's "Reveal":** If Monty still goes through the motions of opening a door, it's incon

sequential.  You already know where the car is.

3. **Switching Option:**  You are offered the chance to switch.  **There is absolutely no benefit to switching.**

    * **If you initially picked the car door (which you would):** Switching would mean moving to a door with a goat,

 guaranteeing you lose.
    * **You would never intentionally pick a goat door initially:** Because you can see everything.

**Probability of Winning:**

* **With transparent doors and rational play:** Your probability of winning becomes **1 (or 100%)**.  You will simply choose the door with the car from

 the beginning.

**In Summary:**

Transparent doors completely alter the Monty Hall Problem because they eliminate the element of uncertainty and hidden information that makes the original problem a paradox.  The game becomes a straightforward exercise in observation. There's no strategy involved, and the decision to switch or stay becomes irrelevant because you already have

 perfect information.

The core of the Monty Hall Problem is about updating probabilities based on new information provided by Monty's action.  Transparent doors remove the need for any probability calculations or strategic thinking because all the information is available upfront.

## Thinking Model examples

The following examples are some complex tasks that require multiple rounds of strategizing and iteratively solving.

### **Example 1**: Code simplification

First, try with a simple code comprehension and simplification example.

In [8]:
prompt = """
How can I simplify this?
`(Math.round(radius/pixelsPerMile * 10) / 10).toFixed(1);`
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=config,
)

print_thoughts(response)

## Answer:
Let's break down this JavaScript code and see how we can simplify it while maintaining its functionality.

**Understanding the Original Code**

The code is performing the following steps:

1. **`radius / pixelsPerMile`**:  This calculates a ratio.  Assuming `radius` is in pixels and `pixelsPerMile` is the number of pixels representing one mile, this division likely converts the radius from pixels to miles.

2. **`* 10`**: This multiplies the result by 10. This is done to shift the decimal point one place to the right, preparing for rounding to the nearest tenth.

3. **`Math.round(...)`**: This rounds the result to the nearest integer.  After multiplying by 10, rounding to the nearest integer effectively rounds the original value to the nearest tenth.

4. **`/ 10`**: This divides the rounded result by 10, shifting the decimal point back to its original position.  This completes the rounding to one decimal place.

5. **`.toFixed(1)`**: This converts the rounded number to a string, ensuring it always has exactly one digit after the decimal point. Even if the rounded number is a whole number (e.g., 2), `toFixed(1)` will make it `"2.0"`.

**Simplifying the Code**

The core logic is already quite concise.  There isn't a significantly shorter way to achieve the *exact* same numerical rounding and string formatting in standard JavaScript. However, we can improve readability and potentially make it slightly more efficient by:

**1.  Using a Variable for Clarity:**

   If you're using this calculation in multiple places or want to make the code easier to understand at a glance, you can introduce a variable to store the intermediate value before formatting:

   ```javascript
   const milesRadius = radius / pixelsPerMile;
   const roundedMilesString = (Math.round(milesRadius * 10) / 10).toFixed(1);
   ```

   This breaks the calculation into two steps and makes it clearer what's being calculated (radius in miles) and what's being done (rounding and formatting).

**2.  Slightly More Concise (but potentially less readable for some):**

   You could combine the rounding and `toFixed` a bit closer, but it might not be significantly more readable:

   ```javascript
   const milesRadius = radius / pixelsPerMile;
   const formattedMilesRadius = (Math.round(milesRadius * 10) / 10).toFixed(1);
   ```

   This is very similar to your original code, just with a variable for the initial division.

**3.  No Significant Numerical Simplification:**

   There isn't a built-in JavaScript `Math` function that directly rounds to a specific number of decimal places *and* returns a number (without converting to a string). The `Math.round(x * 10) / 10` pattern is a common and efficient way to round to one decimal place in JavaScript.

**Which Simplification to Choose?**

* **For Readability and Maintainability:**  The first option (using `milesRadius` variable) is generally the best. It clearly separates the unit conversion from the rounding and formatting. This makes the code easier to understand and modify later.

* **For Conciseness (Slightly):** The second option is slightly more concise but offers minimal readability improvement over the original if you are already familiar with the rounding pattern.

* **No need to over-simplify numerically:**  The core numerical logic (`Math.round(x * 10) / 10`) is already a very efficient and standard way to round to one decimal place in JavaScript when you need to round to the nearest tenth.

**Example with Variable (Recommended):**

```javascript
const radius = 1500; // Example radius in pixels
const pixelsPerMile = 5280; // Example: 5280 pixels per mile (adjust based on your pixel density)

const milesRadius = radius / pixelsPerMile;
const roundedMilesString = (Math.round(milesRadius * 10) / 10).toFixed(1);

console.log("Radius in pixels:", radius);
console.log("Pixels per mile:", pixelsPerMile);
console.log("Radius in miles (rounded to one decimal place):", roundedMilesString);
```

**In summary, while the original code is already fairly concise, adding a variable to represent the intermediate value (`milesRadius`) significantly improves readability and maintainability without sacrificing efficiency.  There isn't a drastically shorter or numerically simpler way to achieve the exact same result in standard JavaScript.**

As a comparison here's what you'd get with the [Gemini 2.0 Flash](https://cloud.google.com/vertex-ai/generative-ai/docs/gemini-v2) base model.

Unlike thinking mode, the normal model does not articulate its thoughts and tries to answer right away which can lead to more simple answers to complex problems.

In [9]:
response = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents=prompt,
)

display(Markdown(response.text))

Okay, let's break down how to simplify this JavaScript code snippet. The goal is to make it more readable and potentially more efficient while achieving the same result.

**Understanding the Current Code**

The code is designed to:

1.  **`radius / pixelsPerMile`**:  Divide the `radius` value by `pixelsPerMile`.  This likely represents converting a radius from some unit (like miles) to pixels.

2.  **`* 10`**: Multiply the result by 10.

3.  **`Math.round(...)`**: Round the result to the nearest whole number.

4.  **`/ 10`**: Divide the rounded result by 10 again.  This effectively rounds the original result to the nearest tenth (one decimal place).

5.  **`.toFixed(1)`**: Converts the number to a string, guaranteeing that it has exactly one digit after the decimal point, even if that means adding a trailing zero (e.g., 5 becomes "5.0").

**Potential Simplifications and Considerations**

Here's a breakdown of simplification options, along with explanations:

*   **Combine Multiplication and Division:**

    You can combine the `* 10` and `/ 10` operations into a single multiplication by the appropriate power of 10. In this case they cancel each other out, but this is more for general purposes when the values are different.
    *   Original: `(Math.round(radius/pixelsPerMile * 10) / 10).toFixed(1);`
    *   Simplified: `(Math.round((radius/pixelsPerMile) * 10) / 10).toFixed(1);`

*   **Direct Rounding to One Decimal Place:**

    JavaScript has a more direct way to round to a specific number of decimal places using the `Math.round()` function with some clever multiplication and division.  This avoids some of the potential floating-point precision issues.

    *   Original: `(Math.round(radius/pixelsPerMile * 10) / 10).toFixed(1);`
    *   Simplified:  `(Math.round((radius / pixelsPerMile) * 10) / 10).toFixed(1);`

*   **Simplify with a function**

   If the goal is reusability and understandability, create a function for this:

   ```javascript
   function roundToNearestTenth(value) {
       return (Math.round(value * 10) / 10).toFixed(1);
   }

   const result = roundToNearestTenth(radius / pixelsPerMile);
   ```

**Why these are simplifications:**

*   **Readability:** Using a function makes the purpose of the code very clear. It's self-documenting.

*   **Maintainability:** If you need to change the rounding logic, you only need to modify it in one place (the function).
**Example**

```javascript
const radius = 1234;
const pixelsPerMile = 100;

// Original way
const originalResult = (Math.round(radius / pixelsPerMile * 10) / 10).toFixed(1);
console.log("Original Result:", originalResult); // Output: 123.4

// Simplified way with function
function roundToNearestTenth(value) {
    return (Math.round(value * 10) / 10).toFixed(1);
}

const simplifiedResult = roundToNearestTenth(radius / pixelsPerMile);
console.log("Simplified Result:", simplifiedResult); // Output: 123.4
```

**Important Considerations:**

*   **Floating-Point Precision:**  JavaScript uses floating-point numbers, which can sometimes lead to small rounding errors.  The `toFixed(1)` method will truncate or round the number as necessary to ensure it has the specified number of decimal places, mitigating some of these issues.  If you need extremely precise calculations (e.g., in financial applications), consider using a library designed for arbitrary-precision arithmetic.

*   **Context:** The "best" simplification depends on the overall context of your code.  If you're doing this rounding operation in many places, a function is definitely the way to go.  If it's only used once, the in-line simplified version might be sufficient.

In summary, the function approach is generally the best for readability and maintainability, but the simplified in-line approaches can be useful for one-off cases where you want to reduce the amount of code.  Always prioritize code clarity and maintainability.


### **Example 2**: Geometry problem (with image)

This geometry problem requires complex reasoning and is also using Gemini multimodal capabilities to read the image.

In [ ]:
image_file_path = "generativeai-downloads/images/geometry.png"
image_file_uri = f"gs://{image_file_path}"
image_file_url = f"https://storage.googleapis.com/{image_file_path}"

display(Image(url=image_file_url, width=400))

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(image_file_uri, mime_type="image/png"),
        "What's the area of the overlapping region?",
    ],
    config=config,
)

print_thoughts(response)

### **Example 3**: Mathematical brain teaser


In [ ]:
prompt = """Add mathematical operations (additions, subtractions, multiplications)
to get 746 using these numbers only once: 8, 7, 50, and 4
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=config,
)

print_thoughts(response)

### **Example 4**: Generating question for a specific level of knowledge

This time, the questions require a few types of knowledge, including what is relevant to the [Physics C: Mechanics exam](https://apcentral.collegeboard.org/courses/ap-physics-c-mechanics/exam). The questions generated are not the interesting part, but the reasoning to come up with them shows they are not just randomly generated.

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="Give me a practice question I can use for the AP Physics C: Mechanics exam?",
    config=config,
)

print_thoughts(response)

### **Example 5**: Statistics

Here's a new mathematical problem. Once again, what's interesting is not the answer (as you might know it already) but how the model is coming up with it.

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="You repeatedly flipped a coin until you either flip three heads, or heads tails heads. Which is more likely to happen first?",
    config=config,
)

print_thoughts(response)

### **Example 6**:  Brain teaser with a twist

Here's another brain teaser based on an image, this time it looks like a mathematical problem, but it cannot actually be solved mathematically. If you check the thoughts of the model you'll see that it will realize it and come up with an out-of-the-box solution.

In [ ]:
image_file_path = "generativeai-downloads/images/pool.png"
image_file_uri = f"gs://{image_file_path}"
image_file_url = f"https://storage.googleapis.com/{image_file_path}"

display(Image(url=image_file_url, width=400))

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(image_file_uri, mime_type="image/png"),
        "How do I use three of the pool balls to sum up to 30?",
    ],
    config=config,
)

print_thoughts(response)

## Next Steps

- Explore the Vertex AI [Cookbook](https://cloud.google.com/vertex-ai/generative-ai/docs/cookbook) for a curated, searchable gallery of notebooks for Generative AI.
- Explore other notebooks and samples in the [Google Cloud Generative AI repository](https://github.com/GoogleCloudPlatform/generative-ai).